In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.schema import Document
import os
import shutil


In [4]:

CHROMA_PATH="chroma_1000_50_all-MiniLM-L6-v2"
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



/tmp/ipykernel_2195063/2581603812.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/bjtuzky/.conda/envs/rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:

# 加载文档
loader = TextLoader('term.txt')


def load_documents():
    # loader = TextLoader(DATA_PATH)
    # documents = loader.load()
    # return documents
    loaders = [TextLoader('./term.txt', encoding='utf-8'),TextLoader('./corpus.txt', encoding='utf-8'),TextLoader('./law_explanation.txt', encoding='utf-8')]
    docs = []
    for loader in loaders:
        pages = loader.load()
        docs.extend(pages)
    return docs


def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=50,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[10]
    print(document.page_content)
    print(document.metadata)

    return chunks

documents = load_documents()
chunks = split_text(documents)

# 加载嵌入模型


if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)
# 创建数据库
db = Chroma.from_documents(chunks, embeddings,persist_directory=CHROMA_PATH)

# 保存数据库到本地
db.persist()


Split 3 documents into 9617 chunks.
（十二）诈骗罪

1.构成诈骗罪的，根据下列情形在相应的幅度内确定量刑起点：

（1）达到数额较大起点的，在一年以下有期徒刑、拘役幅度内确定量刑起点。

（2）达到数额巨大起点或者有其他严重情节的，在三年至四年有期徒刑幅度内确定量刑起点。

（3）达到数额特别巨大起点或者有其他特别严重情节的，在十年至十二年有期徒刑幅度内确定量刑起点。依法应当判处无期徒刑的除外。

2.在量刑起点的基础上，根据诈骗数额等其他影响犯罪构成的犯罪事实增加刑罚量，确定基准刑。

3.构成诈骗罪的，根据诈骗的数额、手段、危害后果等犯罪情节，综合考虑被告人缴纳罚金的能力，决定罚金数额。

4.构成诈骗罪的，综合考虑诈骗的起因、手段、数额、危害后果、退赃退赔等犯罪事实、量刑情节，以及被告人的主观恶性、人身危险性、认罪悔罪表现等因素，决定缓刑的适用。对实施电信网络诈骗的，从严把握缓刑的适用。

（十三）抢夺罪

1.构成抢夺罪的，根据下列情形在相应的幅度内确定量刑起点：

（1）达到数额较大起点或者二年内三次抢夺的，在一年以下有期徒刑、拘役幅度内确定量刑起点。

（2）达到数额巨大起点或者有其他严重情节的，在三年至五年有期徒刑幅度内确定量刑起点。

（3）达到数额特别巨大起点或者有其他特别严重情节的，在十年至十二年有期徒刑幅度内确定量刑起点。依法应当判处无期徒刑的除外。

2.在量刑起点的基础上，根据抢夺数额、次数等其他影响犯罪构成的犯罪事实增加刑罚量，确定基准刑。

多次抢夺，数额达到较大以上的，以抢夺数额确定量刑起点，抢夺次数可以作为调节基准刑的量刑情节；数额未达到较大的，以抢夺次数确定量刑起点，超过三次的次数作为增加刑罚量的事实。

3.构成抢夺罪的，根据抢夺的数额、次数、手段、危害后果等犯罪情节，综合考虑被告人缴纳罚金的能力，决定罚金数额。

4.构成抢夺罪的，综合考虑抢夺的起因、数额、手段、次数、危害后果、退赃退赔等犯罪事实、量刑情节，以及被告人的主观恶性、人身危险性、认罪悔罪表现等因素，决定缓刑的适用。

（十四）职务侵占罪

1.构成职务侵占罪的，根据下列情形在相应的幅度内确定量刑起点：

（1）达到数额较大起点的，在一年以下有期徒刑、拘役幅度内确定量刑起点。

（2）达到数额巨大起点的，在三年至四年有期徒刑幅度

In [3]:

db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)

query = """最高人民法院关于处理自首和立功"""
results = db.similarity_search_with_relevance_scores(query, k=10)

# 打印检索结果
cnt=0
for doc, score in results:
    # if query not in doc.page_content:
    #     continue
    print(f"Content: {doc.page_content}")
    print(f"Relevance Score: {score}")
    print("-----")
    cnt+=1
    if cnt==10:
        break


NameError: name 'CHROMA_PATH' is not defined